In [1]:
import threading
from IPython.display import display
import ipywidgets as widgets

class Animation:
    _thread = None
    _running_event = threading.Event()

    def __init__(self, canvas, update):
        self.canvas = canvas
        self.update = update
        self._create_styles()
        self._create_ui()

    def run(self):
        while Animation._running_event.is_set():
            self.update()

    def start(self):
        if not Animation._running_event.is_set():
            Animation._running_event.set()
            Animation._thread = threading.Thread(target=self.run)
            Animation._thread.start()

    def stop(self):
        Animation._running_event.clear()
        if Animation._thread is not None:
            Animation._thread.join()

    def display(self):
        container = widgets.VBox([self.styles, self.ui])
        display(container)

    def _create_ui(self):
        self.play_pause_button = widgets.ToggleButton(
            value=True,
            button_style='',
            tooltip='Play/Pause',
            icon='pause',
            _dom_classes=['play-pause-button']
        )

        self.play_pause_button.observe(self._toggle_play_pause, 'value')
        self.controls = widgets.HBox(
            [self.play_pause_button],
            _dom_classes=['controls']
        )

        self.ui = widgets.VBox([self.canvas, self.controls])
    
    def _toggle_play_pause(self, change):
        if change['new']:
            self.play_pause_button.icon = 'pause'
            self.start()
        else:
            self.play_pause_button.icon = 'play'
            self.stop()

    def _create_styles(self):
        css = """
            canvas {
                display: block;
                margin: 0 auto !important;
                padding: 0;
                background: white;  /* Ensure the canvas has a background color */
            }

            canvas:focus {
                outline: none;
                border: none;
            }

            .cell-output-ipywidget-background {
                width: auto !important;
                padding: 0 !important;
                background: transparent !important;
            }
     
            i.fa {
                margin: 0 !important;
            }

            .play-pause-button {
                width: 50px;
                height: 50px;
                border-radius: 100%;
                padding: 0;
            }
 
            .play-pause-button:focus {
                outline: none !important;
                box-shadow: none !important;
            }

            .controls {
                margin: 10px 0;
                width: 100%;
                display: flex;
                justify-content: center;
            }
        """

        self.styles = widgets.HTML(f"<style>{css}</style>")


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from ipycanvas import Canvas, hold_canvas
import time
from library import Particle
import random

G = 0.2

canvas = Canvas(width=400, height=400)

particles = []

for i in range(50):
    x = random.randint(0, 400)
    y = random.randint(0, 400)
    vx = random.uniform(-1, 1)
    vy = random.uniform(-1, 1)
    radius = random.randint(1, 20)
    p = Particle(x, y, vx, vy, radius, "orange")
    particles.append(p)


def evolve(dt):
    for p1 in particles:
        for p2 in particles:
            d = ((p1.x - p2.x)**2 + (p1.y - p2.y)**2)**(1/2)
            if d > 4:
                F = G * p1.mass * p2.mass / d**2
                rx = (p1.x - p2.x)/d 
                ry = (p1.y - p2.y)/d
                Fx = -F * rx
                Fy = -F * ry
                a1x = Fx / p1.mass
                a1y = Fy / p1.mass
                p1.vx = p1.vx + a1x * dt
                p1.vy = p1.vy + a1y * dt


    for p in particles:
        p.move(dt)
  
        
def draw():
    with hold_canvas(canvas):
        canvas.fill_style = "black"
        canvas.fill_rect(0, 0, 400, 400)
        for p in particles:
            p.draw(canvas)
            
def update():
    evolve(0.5)
    draw()
    time.sleep(1/30)


anim = Animation(canvas, update)
anim.display()
anim.start()